# 前提環境

公式: https://mamba.readthedocs.io/

Fabric のインストール

In [2]:
!pip install fabric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 11.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 11.2 MB/s eta 0:00:00 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


## Fabric ローカル実行

In [32]:
from fabric import task
from invoke.context import Context
import shutil

@task
def CheckInstallation(c, command):
    result = shutil.which(command)
    print('=== fabfile.main.CheckInstallation()')
    print(f'--> shutil.which: {result}')
    
    return not result is None

CheckInstallation(Context(), 'rustup')

=== fabfile.main.CheckInstallation()
--> shutil.which: None


False

## Mamba のインストール

In [1]:
!pwd

/home/tomoyan/my_projects/jupyter-binder/fabric_example


In [8]:
%%file ./fabfile/install_mamba.py
from fabric import task
from invoke.watchers import Responder

from . import CheckInstallation, tmux_run

@task
@tmux_run
def Mamba_CheckInstalled(c):
    return CheckInstallation(c, 'micromamba')

@task
@tmux_run
def Mamba_Install(c):
    if Mamba_CheckInstalled(c):
        print('Mamba はインストール済みです😉');
        return False
    
    enterResponder = Responder(
        pattern=r'\n>',
        response='\n'
    )
    result = c.run(
        # install.sh が shell を判定するので | bash である必要がある🤔
        #"curl --proto '=https' --tlsv1.2 -sSf https://micro.mamba.pm/install.sh | sh",
        "curl --proto '=https' --tlsv1.2 -sSf https://micro.mamba.pm/install.sh | bash",
        echo=True, hide=False, pty=True,
        watchers=[enterResponder]
    )
    #print(result.stdout.strip())

@task
@tmux_run
def Mamba_Uninstall(c):
    if not Mamba_CheckInstalled(c):
        print('Mamba はインストールされていません🤔');
        return False
    
    yesResponder = Responder(
        pattern=r'Continue\? \(y/N\)',
        response='y\n'
    )
    result = c.run(
        "rustup self uninstall",
        echo=True, pty=True,
        watchers=[yesResponder])


Overwriting ./fabfile/install_mamba.py


In [1]:
!fab --list

/bin/bash: 行 1: fab: コマンドが見つかりません


In [9]:
!fab Mamba-CheckInstalled --debug

=== fabfile.main.CheckInstallation()
--> shutil.which(micromamba): /home/tomoyan/.local/bin/micromamba


In [4]:
!fab Mamba-Install --debug

=== fabfile.main.CheckInstallation()
--> shutil.which(micromamba): None
curl --proto '=https' --tlsv1.2 -sSf https://micro.mamba.pm/install.sh | bash
bin/micromamba
Modifying RC file "/home/tomoyan/.bashrc"
Generating config for root prefix "/home/tomoyan/micromamba"
Setting mamba executable to: "/home/tomoyan/.local/bin/micromamba"
Adding (or replacing) the following in your "/home/tomoyan/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba init' !!
export MAMBA_EXE="/home/tomoyan/.local/bin/micromamba";
export MAMBA_ROOT_PREFIX="/home/tomoyan/micromamba";
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    if [ -f "/home/tomoyan/micromamba/etc/profile.d/micromamba.sh" ]; then
        . "/home/tomoyan/micromamba/etc/profile.d/micromamba.sh"
    else
        export  PATH="/home/tomoyan/micromamba/bin:$PATH"  # extra space after export preve

## micromamba の動作確認

In [11]:
!micromamba --version

1.4.2


In [10]:
!micromamba --help

Version: 1.4.2

Usage: micromamba [OPTIONS] [SUBCOMMAND]

Options:
  -h,--help                   Print this help message and exit
  --version                   


Configuration options:
  --rc-file TEXT ...          Paths to the configuration files to use
  --no-rc                     Disable the use of configuration files
  --no-env                    Disable the use of environment variables


Global options:
  -v,--verbose                Set verbosity (higher verbosity with multiple -v, e.g. -vvv)
  --log-level ENUM:value in {critical->5,debug->1,error->4,info->2,off->6,trace->0,warning->3} OR {5,1,4,2,6,0,3}
                              Set the log level
  -q,--quiet                  Set quiet mode (print less output)
  -y,--yes                    Automatically answer yes on prompted questions
  --json                      Report all output as json
  --offline                   Force use cached repodata
  --dry-run                   Only display what would have been done
  --downlo

## Mamba のアンインストール

In [38]:
!fab Rust-Uninstall

which: None
Rust はインストールされていません🤔


# 参考文献
https://docs.pyinvoke.org/en/stable/concepts/watchers.html<br>
https://blog.masu-mi.me/post/2021/02/14/catch-up-fabric2/<br>
https://docs.python.org/ja/3/library/inspect.html<br>
https://stackoverflow.com/questions/26515595/how-does-one-ignore-unexpected-keyword-arguments-passed-to-a-function<br>